In [2]:
#importar bibliotecas
import dash
from dash import dcc
from dash import html

import plotly.graph_objs as go

import pandas as pd
import math

from dash.dependencies import Input, Output
from datetime import datetime
from sqlalchemy import create_engine

In [3]:
#FUNÇÃO LINEAR

# define layout
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(children=[  

    # cabeçalho
    html.H1(children = 'Linear function', style={'color':'red'}),    

    # plot 
    dcc.Graph(
        figure = {
                'data': [go.Scatter(x = pd.Series(range(-100, 100, 1)), 
                                    y = pd.Series(range(-100, 100, 1)), 
                                    mode = 'lines',
                                    name = 'linear_func')],
                'layout': go.Layout(xaxis = {'title': 'x'},
                                    yaxis = {'title': 'y'})
             },      
        id = 'linear_func_id'
    ),         
  
])
# lógica do dashboard
if __name__ == '__main__':
    app.run_server(debug=True)

In [4]:
#SENO E COSSENO COM BOTAO

# defina o layout
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(
    children=[

        # usando html para criar o cabeçalho
        html.H1(children='Trigonometric functions', style={'color':'red'}),

        #usando html para criar o botão
        html.Label('Display Mode', style={'color':'red'}),
        dcc.RadioItems(

            #opções do que cada botao vem escrito e de qual o valor dele
            options=[
                {'label':'Display sin(x)', 'value':'sin'},
                {'label':'Display cos(x)', 'value':'cos'},
            ],

            #valor inicial
            value = 'cos',

            #nome do botao
            id = 'mode_selector',

            #alinhando os botoes na horizontal
            inline=True,
        ),

        #o botao chama o gráfico
        dcc.Graph(
            id = 'trig_func'
        ),

    ]
)

#lógica do dashboard
@app.callback(
    [
        Output('trig_func', 'figure'),
    ],
    [
        Input('mode_selector', 'value'),
    ],
)


def update_figures(selected_mode):
    
    #gerando os valores x e y para os dados da função
    x = range(-100, 100, 1)
    x = [x / 10 for x in x]
    y = [math.sin(x) for x in x]
    if selected_mode == 'cos':
        y = [math.cos(x) for x in x]
    

    #gerar o gráfico que sera parte do retorno da funcao
    data = [
        go.Scatter(
            x=pd.Series(x), 
            y=pd.Series(y), 
            mode='lines', 
            name=selected_mode + '(x)',
        ),
    ]   


    #retorno da função
    return(
        {
            'data' : data,
            'layout' : go.Layout(xaxis={'title': 'x'}, yaxis={'title': 'y'})
        },
    )



# lógica do dashboard
if __name__ == '__main__':
    app.run_server(debug=True)

In [5]:
# defina os dados a serem exibidos
games_raw = pd.read_csv('datasets/vgsales.csv')

games_raw = games_raw.rename(columns={'Year':'Year_of_Release'})

games_raw = games_raw.dropna(subset=['Year_of_Release'])
games_raw['Year_of_Release'] = pd.to_datetime(games_raw['Year_of_Release'].astype(int).astype(str), format='%Y')

games_raw.columns = games_raw.columns.str.lower()
games_raw

,rank,name,platform,year_of_release,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,1,Wii Sports,Wii,2006-01-01,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985-01-01,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008-01-01,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009-01-01,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996-01-01,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002-01-01,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003-01-01,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008-01-01,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010-01-01,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [6]:
#gráfico de area empilhada
# defina os dados para o relatório
games_grouped = (
    games_raw.groupby(['genre', 'year_of_release'])
    .agg({'name': 'count'})
    .reset_index()
    .rename(columns={'name': 'games_released'})
)

# defina os gráficos a serem exibidos
data = []
for genre in games_grouped['genre'].unique():
    current = games_grouped.query('genre == @genre')
    data += [
        go.Scatter(
            x=current['year_of_release'],
            y=current['games_released'],
            mode='lines',
            stackgroup='one',
            name=genre,
        )
    ]
# defina o layout
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(
    __name__, external_stylesheets=external_stylesheets, compress=False
)
app.layout = html.Div(
    children=[
        # html
        html.H1(children='Games released by year'),
        dcc.Graph(
            figure={
                'data': data,
                'layout': go.Layout(
                    xaxis={'title': 'Year'}, yaxis={'title': 'Games released'}
                ),
            },
            id='games_by_year',
        ),
    ]
)

# lógica do dashboard
if __name__ == '__main__':
    app.run_server(debug=True)

In [8]:
#filtrando df para dashboard
agg_games_year_genre_platform = games_raw.groupby(['year_of_release','genre','platform']).agg({
    'name':'sum',
    'global_sales':'sum'
}).rename(columns={'name':'games'}).reset_index()

agg_games_year_genre_platform.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1804 entries, 0 to 1803
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   year_of_release  1804 non-null   datetime64[ns]
 1   genre            1804 non-null   object        
 2   platform         1804 non-null   object        
 3   games            1804 non-null   object        
 4   global_sales     1804 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 70.6+ KB


<h2 style = "color:red;">Desenvolvimento do Dashboard e o básico de composição</h2>

In [11]:
#games_raw - é o df original

#anotação descrevendo o dashboard
note = '''
          Este dashboard mostra o histórico do mercado de jogos (dispositivos móveis excluídos).
				  Use seletores de intervalo de tempo, gênero e plataformas para filtrar o dashboard.
				  Use o seletor de modo de exibição para alternar entre valores absolutos e relativos.
       '''


#definindo o layout
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(
    __name__, external_stylesheets=external_stylesheets, compress=False
)
app.layout = html.Div(
    children=[

        #cabeçalho
        html.H1(children='Histórico do mercado de jogos', style={'color':'red'}),
        #pulando linhas para organizar
        html.Br(),
        #primeira linha de gráficos
        html.Div(
            [
                #primeiro gráfico da primeira linha
                html.Div(
                    [
                        #grafico com botão para escolha de opção unica:
                        #no caso escolher se os dados serão apresentados com valores absolutos ou relativos
                        html.Label('Jogos lançados por gênero:'),

                        #botões de opção unica
                        dcc.RadioItems(
                            options=[
                                {
                                    'label':'Valores absolutos',
                                    'value':'absolute_values',
                                },
                                {
                                    'label':'% do número total de jogos lançados',
                                    'value':'relative_values',
                                },
                            ],
                            value='absolute_values',
                            id='mode_selector',
                        ),

                        #gráfico
                        dcc.Graph(id='launches_by_year')
                    ],
                    #tamanho do gráfico, lembrando que o total da linha soma doze. Faltam quatro.
                    className='eight columns',
                ),

                #segundo gráfico da primeira linha
                html.Div(
                    [
                        #jogos 
                        html.Label('Jogos lançados por plataforma:'),
                        html.Br(),
                        html.Br(),
                        dcc.Graph(id='launches_by_platform'),
                    ],
                    #esse segundo gráfico ocupa o resto do espaço na linha.
                    className= 'four columns',
                ),
            ],
            #todos os gráficos aqui dentro desse Div pertencem a mesma linha
            className='row'
        ),

        #segunda linha
        html.Div(
            [
                #primeiro gráfico da segunda linha
                html.Div(
                    [
                        #
                        html.Label('Jogos vendidos por gênero:'),
                        #chamando o gráfico
                        dcc.Graph(id='sales_by_year'),
                    ],
                    #esse gráfico ocupa 8 de 12 espaços na linha
                    className='eight columns',
                ),

                #segundo gráfico da segunda linha
                """html.Div(
                    [
                        #
                        html.Label('Pontuações médias por gênero:'),
                        #
                        dcc.Graph(id='score_scatter'),
                    ],
                    #ocupando o resto da linha
                    className='four columns',
                ),"""
            ],
            #os graficos desse Div pertencem a mesma linha
            className='row',
        ),

        #descrição
        html.Label(children=note, style={'color':'yellow'}),
        #pulando linha
        html.Br(),

        #terceira linha de gráficos
        html.Div(
            [
                #primeiro botão da terceira linha
                html.Div(
                    [
                        #Título
                        html.Label("Ano de lançamento:"),
                        #botao
                        dcc.DatePickerRange(
                            #data padrão do inicio do filtro
                            start_date=agg_games_year_genre_platform['year_of_release'].dt.date.min(),
                            #data padrão do fim do filtro
                            end_date=datetime(2016,1,1).strftime('%Y-%m-%d'),
                            #O mês inicial que será exibido ao abrir o seletor de datas.
                            initial_visible_month=datetime(2016,1,1).strftime('%Y-%m-%d'),
                            #Formato em que as datas são exibidas.
                            display_format='YYYY-MM-DD',
                            id='dt_selector',
                        ),
                    ],
                    className='two columns',
                ),

                #segundo botão da terceira linha
                html.Div(
                    [
                        #Título
                        html.Label('Gêneros:'),
                        #botão
                        dcc.Dropdown(

                            #lista de opções a serem exibidas. Cada uma tem label e value
                            options=[
                                {'label':x, 'value':x}
                                for x in agg_games_year_genre_platform['genre'].unique()
                            ],
                            #defini os valores que começam selecionados, nesse caso todos estao selecionados
                            value=agg_games_year_genre_platform['genre'].unique().tolist(),
                            #multiplas opções que podem ser escolhidas
                            multi=True,
                            #id do botao
                            id='genre_selector',
                        ),
                    ],
                    className='four columns',
                ),

                #terceiro botão da terceira linha
                html.Div(
                    [
                        #título
                        html.Label('Plataformas:'),

                        #botão
                        dcc.Dropdown(
                            options=[
                                {'label':x, 'value':x}
                                for x in agg_games_year_genre_platform['platform'].unique()
                            ],
                            value=agg_games_year_genre_platform['platform'].unique().tolist(),
                            multi=True,
                            id='platform_selector',
                        ),
                    ],
                    className='six columns',
                ),

            ],
            className='row',
        ),

    ]
)


# lógica do dashboard
@app.callback(
    #Output
    [
      Output('launches_by_year','figure'),
      Output('launches_by_platform','figure'),
      Output('sales_by_year', 'figure'),
      #Output('score_scatter', 'figure'),
    ],
    #Input
    [
      Input('dt_selector', 'start_date'),
      Input('dt_selector', 'end_date'),
      Input('mode_selector', 'value'),
      Input('genre_selector', 'value'),
      Input('platform_selector', 'value'),
    ]
)

def update_figures(start_date, end_date, mode, selected_genres, selected_platforms):
    
    #aplicando filtros de data
    filtered = agg_games_year_genre_platform.query('year_of_release >= start_date and year_of_release <= end_date')

    #aplicando filtro de genero
    filtered = filtered.query('genre in selected_genres')

    #aplicando filtro de plataforma
    filtered = filtered.query('platform in selected_platforms')
    
    #df com a somatoria de vendas e jogos vendidos por gênero filtrado
    games_by_genre = (
        filtered.groupby(['year_of_release', 'genre'])
        .agg({'games':'sum', 'global_sales':'sum'})
        .reset_index()
    )

    #df com a somatoria de jogos produzidos por plataforma filtrada
    games_by_platform = (
        filtered.groupby(['platform'])
        .agg({'games':'sum'})
        .reset_index()
    )

    #calculando em porcentagem com relacao a todos os jogos vendidos
    games_by_platform['percent'] = games_by_platform['games'] / games_by_platform['games'].sum()
    #renomeando as plataformas que representam menos de 5% dos jogos criados como outros
    games_by_platform['games'].loc[games_by_platform['percent'] < 0.05, 'platform'] = 'Other'
    #agrupando todas essas plataformas numa só
    games_by_platform = (games_by_platform.groupby(['platform']).agg({'games':'sum'}).reset_index())

    y_label = 'Jogos lançados'
    y_label_sales = 'Jogos vendidos, milhões de cópias'
    if mode == 'relative_values':

        y_label = '% de jogos vendidos'

        y_label_sales = '% de jogos vendidos'

        total = (
                games_by_platform.groupby()
                .agg()
                .rename(columns={})
                )
        
        games_by_genre = (
            games_by_genre.set_index('year_of_release')
            .join(total)
            .reset_index()
        )

        games_by_genre['games'] = (
            games_by_genre['games'] / games_by_genre['total_launches']
            )
        
        games_by_genre['total_copies_sold'] = (
            games_by_genre['total_copies_sold'] / games_by_genre['total_copies_sold']
        )




if __name__ == '__main__':
    app.run_server(debug=True)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File c:\Users\Gustavo\anaconda3\envs\projeto\Lib\site-packages\pandas\core\computation\scope.py:231, in Scope.resolve(
    self=Scope(scope=['Timestamp',
 'datetime',
 'True',
...genre',
 'platform',
 'games',
 'global_sales']
),
    key='start_date',
    is_local=False
)
    230 if self.has_resolvers:
--> 231     return self.resolvers[key]
        self = Scope(scope=['Timestamp',
 'datetime',
 'True',
 'False',
 'list',
 'tuple',
 'inf',
 'Inf',
 '__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__builtin__',
 '__builtins__',
 '_ih',
 '_oh',
 '_dh',
 'In',
 'Out',
 'get_ipython',
 'exit',
 'quit',
 'open',
 '_',
 '__',
 '___',
 '__vsc_ipynb_file__',
 '_i',
 '_ii',
 '_iii',
 '_i1',
 'note',
 'external_stylesheets',
 '_i2',
 'dash',
 'dcc',
 'html',
 'go',
 'pd',
 'math',
 'Input',
 'Output',
 'create_engine',
 '_i3',

In [14]:
agg_games_year_genre_platform.head()

,year_of_release,genre,platform,games,global_sales
0,1980-01-01,Action,2600,Freeway,0.34
1,1980-01-01,Fighting,2600,Boxing,0.77
2,1980-01-01,Misc,2600,Kaboom!DefenderBridgeCheckers,2.71
3,1980-01-01,Shooter,2600,AsteroidsMissile Command,7.07
4,1980-01-01,Sports,2600,Ice Hockey,0.49
